In [202]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import wandb

In [176]:
timeseries = np.load('data.npy').T

In [177]:
# train-test split for time series
train_size = int(len(timeseries) * 0.8)
test_size = len(timeseries) - train_size
train, test = timeseries[:train_size], timeseries[train_size:]


def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
        
    return torch.tensor(X), torch.tensor(y)

lookback = 50
X_train_idx = np.arange(len(train)-lookback)
y_train_idx = np.arange(lookback, len(train))

X_test_idx = np.arange(len(train), len(timeseries)-lookback)
y_test_idx = np.arange(len(train)+lookback, len(timeseries))

test_idx = np.arange(len(train), len(timeseries))
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)


In [249]:
import slim

In [252]:
class extract_tensor(nn.Module):
    def forward(self,x):
        # Output shape (batch, features, hidden)
        tensor, _ = x
        # Reshape shape (batch, hidden)
        return tensor[:, -1, :]

class AirModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, time_points):
        super().__init__()
        
        self.coeffs = torch.nn.Parameter(torch.tensor(
            np.random.rand(3, time_points), requires_grad=True))
        
        self.F = torch.nn.ParameterList()
        
        for _ in range(3):
            f_i = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True, bias=True, bidirectional=True)
            linear = slim.linear.SpectralLinear(hidden_size*2, output_size, bias=True, sigma_max=1.0, sigma_min=0)
            self.F.append(nn.Sequential(f_i, extract_tensor(), linear, nn.BatchNorm1d(50)))
            

    def forward(self, x, idx):
        #x = self.F[0](x)
        
        x = torch.stack([self.coeffs[i, idx].view(-1,1,1)*f_i(x)  
                          for i, f_i in enumerate(self.F)]).sum(dim=0)
        
        return x
    
def multi_step_loss(X, y, model, loss_fn, lookback, teacher_forcing_ratio=0.5):
    recon = torch.zeros_like(y)
    recon[:lookback] = X[:lookback]  # Set initial inputs from X

    for i in range(len(X) - lookback):
        # Predict next step based on the past 'lookback' steps
        y_pred = model(recon[i:i + lookback].float(), torch.arange(i, i + lookback))
        
        # Update the reconstruction with the new prediction
        if torch.rand(1) < teacher_forcing_ratio:
            recon[i + lookback] = y[i + lookback]  # Use actual next value (ground truth)
        else:
            recon[i + lookback] = y_pred[-1] 
    # Compute the loss over the entire reconstructed sequence
    return loss_fn(recon, y)
    #recon = torch.zeros_like(y)
    #recon[:lookback] = X[:lookback]
    #for i in range(lookback, len(y)):
    #    recon[i] = model(recon[i-lookback:i].float(), torch.arange(i-lookback, i))
        
    #for i in range(len(X)-lookback):
    #    y_pred = model(recon[i:i+lookback].float(), torch.arange(i, i+lookback))
    #    recon[i+lookback] = y_pred[-1]
    #return loss_fn(recon, y)
    #y_pred = model(X)
    #return loss_fn(y_pred, y)
    
    
class TimeSeriesDataset(torch.utils.data.Dataset):
    # TODO: make more explicit
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], idx
    
wandb.login(key='a79ac9d4509caa0d5e477c939a41d790e7711171')


if False:
    run = wandb.init(
        # Set the project where this run will be logged
        project=f"3_State_Bias_C-Init_Rand_A-Init_Rand_SpectralLinear",
        # dir=f'/state_{str(args.num_subdyn)}/fixpoint_change_{str(args.fix_point_change)}', # This is not a wandb feature yet, see issue: https://github.com/wandb/wandb/issues/6392
        # name of the run is a combination of the model name and a timestamp
        # reg{str(round(args.reg, 3))}_
        name=f"smooth_random_coeffs",
        # Track hyperparameters and run metadata
        config={
            "batch_size": 64,
        },
    )

model = AirModel(input_size=4, hidden_size=4, output_size=4, time_points=len(timeseries)).float()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()

#loader = TimeSeriesDataset(data.TensorDataset(X_train.float(), y_train.float()), shuffle=True, batch_size=8)
data = TimeSeriesDataset(list(zip(X_train.float(), y_train.float())))

# create an iterable over our data, no shuffling because we want to keep the temporal information
loader = torch.utils.data.DataLoader(data, batch_size=64, shuffle=True)

initial_teacher_forcing_ratio = 0.5
final_teacher_forcing_ratio = 0.0
n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    for (X_batch, y_batch),idx in loader:
        
        teacher_forcing_ratio = initial_teacher_forcing_ratio - \
                    (initial_teacher_forcing_ratio -
                     final_teacher_forcing_ratio) * (epoch / n_epochs)
        # indices of the batch
        y_pred = model(X_batch.float(), X_train_idx[idx])
        
        coeff_delta = model.coeffs[:,1:] - model.coeffs[:, :-1]
        # L1 norm of the difference
        smooth_reg = coeff_delta.abs().sum()
        smooth_reg_loss = 0.001 * smooth_reg * 4
        
        multi_loss = multi_step_loss(X_batch, y_batch, model, loss_fn, lookback, teacher_forcing_ratio)
        loss = smooth_reg_loss + 0.1*multi_loss + loss_fn(y_pred, y_batch)
        wandb.log({'loss': loss.item()})
        # wandb.log({'sparsity_loss': sparsity_loss.item()})
        wandb.log({'smooth_reg_loss': smooth_reg_loss.item()})
        wandb.log({'multi_reconstruction_loss': multi_loss.item()})
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 7)
        optimizer.step()
        
    # Validation
    if epoch % 10 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train.float(), X_train_idx)
        train_rmse = np.sqrt(loss_fn(y_pred, y_train))
        y_pred = model(X_test.float(), X_test_idx)
        test_rmse = np.sqrt(loss_fn(y_pred, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

with torch.no_grad():
    # shift train predictions for plotting
    train_plot = np.ones_like(timeseries) * np.nan
    y_pred = model(X_train.float(), X_train_idx)
    y_pred = y_pred[:, -1, :]
    train_plot[lookback:train_size] = model(X_train.float(), X_train_idx)[:, -1, :]
    # shift test predictions for plotting
    test_plot = np.ones_like(timeseries) * np.nan
    test_plot[train_size+lookback:len(timeseries)] = model(X_test.float(), X_test_idx)[:, -1, :]
# plot
plt.plot(timeseries)
plt.plot(train_plot, c='r')
plt.plot(test_plot, c='g')
plt.show()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: ERROR Control-C detected -- Run data was not synced


In [240]:
import util
util.plotting(model.coeffs.detach().numpy()[:,:train_size].T, title='coeffs');

In [241]:
time_series, _ = create_dataset(timeseries, lookback=lookback)

In [242]:
# multi-step reconstruction by always using the last prediction as input
recon = torch.zeros_like(time_series)
recon[:lookback] = time_series[:lookback]
with torch.no_grad():
    for i in range(len(time_series)-lookback):
        y_pred = model(recon[i:i+lookback].float(), torch.arange(i, i+lookback))
        recon[i+lookback] = y_pred[-1]
        #recon.append(y_pred[-1].numpy())
#recon = np.array(recon)


In [243]:
# residuals 
residuals = time_series[:, -1, :] - recon.detach().numpy()[:, -1, :]

In [244]:
util.plotting(recon.detach().numpy()[:, -1, :], title='reconstruction', stack_plots=False);

In [245]:
result = model(time_series.float(), torch.arange(len(timeseries)-lookback))
util.plotting([time_series[:, -1, :],result.detach().numpy()[:, -1, :]], title='result', stack_plots=True);

In [246]:
from re import X


X2_hat_multistep = []

X2_hat_multistep[:50] = X_train[:50]

for i in range(50, len(X_train)):
    X2_hat_multistep.append(model(X2_hat_multistep[i-50:i][0].float().unsqueeze(0), X_train_idx[i-50:i])[:, -1, :])
    
X2_hat_multistep = torch.stack(X2_hat_multistep)
    

In [247]:
util.plotting(X2_hat_multistep.detach().numpy()[:, -1, :], title='X2_hat');

In [95]:
X2_hat = model(X_test.float(), X_test_idx)[:, -1, :]

In [36]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
lines = px.line(X2_hat)
for line in lines.data:
    fig.add_trace(line)
lines = px.line(test_plot)
for line in lines.data:
    fig.add_trace(line)
    

In [37]:
fig.show()